
# **Projeto Aplicado IV — Entrega 3**  
**Predição da Taxa de Desemprego no Brasil: Séries Temporais com CAGED, PNAD e SELIC**

> Notebook **executável e reproduzível** com modelagem **SARIMAX** e **LGBM** (holdout dos últimos 4 trimestres), seguindo os requisitos da Etapa 3.


## 1. Setup, seeds e imports

In [ ]:

import warnings, os, contextlib, io, random
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
np.random.seed(42)
random.seed(42)


## 2. Carregamento da base consolidada (RAW do GitHub) + preparação

In [ ]:

# Fonte consolidada (trimestral)
RAW_URL = (
    "https://raw.githubusercontent.com/"
    "fpaterni10/projeto-aplicado-iv-desemprego-br/main/"
    "dataset/tratados/caged_pnad_selic_trimestral.csv"
)

df0 = pd.read_csv(RAW_URL)

# Index trimestral com freq explícita (Q-DEC)
per = pd.PeriodIndex(df0["periodo_tri"].astype(str), freq="Q-DEC")
df = df0.set_index(per).sort_index()
df.index.name = "periodo"
df.index = df.index.asfreq("Q-DEC")

# Coluna de data (fim do trimestre) para gráficos
df["data"] = df.index.to_timestamp(how="end")

# Features básicas
df["caged_roll3"] = df["caged_saldo_tri"].rolling(3, min_periods=3).mean()
df["caged_roll3_asinh"] = np.arcsinh(df["caged_roll3"])

# Dummies sazonais e regime pós-2021
df["quarter"] = df.index.quarter
Q = pd.get_dummies(df["quarter"], prefix="Q", drop_first=False)
df["post_2021"] = (df["data"] >= "2021-01-01").astype(int)

# Base inicial de modelagem
df_model = pd.concat([df, Q], axis=1).copy()

# Lags e derivadas para ML
for k in [1, 2, 3, 4]:
    df_model[f"tx_lag{k}"] = df_model["tx_desocupacao"].shift(k)
df_model["caged_diff1"]      = df_model["caged_saldo_tri"].diff(1)
df_model["selic_diff1"]      = df_model["selic_tri"].diff(1)
df_model["caged_roll3_std"]  = df_model["caged_saldo_tri"].rolling(3).std()

# Remove NaNs de rolling/lag/diff
df_model = df_model.dropna().copy()

print(df_model.info())
df_model.head(8)


## 3. Utilitários: métricas + split temporal (holdout)

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error

def metrics_report(y_true, y_pred, prefix="modelo"):
    """MAE, RMSE e MAPE (%). Compatível com versões antigas do sklearn."""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    n = min(len(y_true), len(y_pred))
    y_true = y_true[:n]; y_pred = y_pred[:n]
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # (sem 'squared' kwarg)
    denom = np.where(np.abs(y_true) < 1e-12, np.nan, y_true)
    mape = np.nanmean(np.abs((y_true - y_pred) / denom)) * 100
    return pd.DataFrame({"MAE":[mae], "RMSE":[rmse], "MAPE (%)":[mape]}, index=[prefix])

def temporal_holdout(df, n_test=4):
    """Últimos n_test trimestres = teste, preservando a ordem temporal."""
    if "data" in df.columns:
        df2 = df.sort_values("data").copy()
    else:
        df2 = df.sort_index().copy()
    return df2.iloc[:-n_test].copy(), df2.iloc[-n_test:].copy()


## 4. Modelo 1 — LGBM Regressor (lags + exógenas; logs silenciosos)

In [ ]:

from lightgbm import LGBMRegressor

# Split temporal
train_lgb, test_lgb = temporal_holdout(df_model, n_test=4)

# Conjunto de features (usa apenas as que existirem em df_model)
features = [
    "caged_roll3_asinh", "caged_saldo_tri", "caged_roll3",
    "caged_diff1", "caged_roll3_std",
    "selic_tri", "selic_diff1",
    "tx_lag1", "tx_lag2", "tx_lag3", "tx_lag4",
    "Q_1", "Q_2", "Q_3", "Q_4", "post_2021"
]
features = [f for f in features if f in df_model.columns]
target = "tx_desocupacao"

X_train, y_train = train_lgb[features], train_lgb[target].to_numpy()
X_test,  y_test  = test_lgb[features],  test_lgb[target].to_numpy()

# Modelo (sem conflitos de alias e com logs silenciosos)
lgbm = LGBMRegressor(
    n_estimators=800, learning_rate=0.03,
    num_leaves=63, max_depth=6,
    min_child_samples=5,          # usar apenas este (alias de min_data_in_leaf)
    subsample=0.9, colsample_bytree=0.9,
    reg_lambda=0.1,
    random_state=42, n_jobs=-1, verbosity=-1
)

# Treino silencioso
with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
    lgbm.fit(X_train, y_train)

pred_lgb = lgbm.predict(X_test)
lgbm_metrics = metrics_report(y_test, pred_lgb, prefix="LGBM (seed=42)")
lgbm_metrics


## 5. Modelo 2 — SARIMAX (1,1,2)×(1,0,1,4) com exógenas do CAGED

In [ ]:

from statsmodels.tsa.statespace.sarimax import SARIMAX

# Mesmo holdout do LGBM para comparação justa
train_sar, test_sar = temporal_holdout(df_model, n_test=4)

# Endógena e exógenas com MESMA frequência trimestral reconhecida
y_train_sar = train_sar["tx_desocupacao"].asfreq("Q-DEC")
y_test_sar  = test_sar["tx_desocupacao"].asfreq("Q-DEC")

exog_cols = ["caged_saldo_tri", "caged_roll3_asinh"]
exog_cols = [c for c in exog_cols if c in df_model.columns]
X_train_sar = train_sar[exog_cols].asfreq("Q-DEC")
X_test_sar  = test_sar[exog_cols].asfreq("Q-DEC")

# Ajuste
sarimax = SARIMAX(
    endog=y_train_sar, exog=X_train_sar,
    order=(1,1,2), seasonal_order=(1,0,1,4),
    enforce_stationarity=False, enforce_invertibility=False
).fit(disp=False, maxiter=500)

# Forecast exatamente do tamanho do holdout, com exógenas alinhadas
pred_sar = sarimax.get_forecast(steps=len(y_test_sar), exog=X_test_sar).predicted_mean

sarimax_metrics = metrics_report(y_test_sar.to_numpy(), pred_sar.to_numpy(),
                                prefix="SARIMAX (1,1,2)(1,0,1,4)")
sarimax_metrics


## 6. Comparativo de métricas e gráfico (holdout)

In [ ]:

# Tabela comparativa
comparativo = pd.concat([lgbm_metrics, sarimax_metrics])
print(comparativo)

# Gráfico Real vs Pred (holdout)
import matplotlib.pyplot as plt

t = test_lgb.index.to_timestamp(how="end") if hasattr(test_lgb.index, "to_timestamp") else test_lgb["data"]
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(t, y_test,     label="Real")
ax.plot(t, pred_lgb,   label="LGBM")
ax.plot(t, pred_sar,   label="SARIMAX")
ax.set_title("Real vs Pred — Holdout trimestral (últimos 4 trimestres)")
ax.legend()
fig.tight_layout()

# Salvar figura para uso no README/relatório
os.makedirs("docs/figuras", exist_ok=True)
fig.savefig("docs/figuras/real_vs_pred_holdout.png", dpi=160, bbox_inches="tight")

plt.show()
plt.close(fig)


## 7. Hiperparâmetros avaliados (resumo)

In [ ]:

hp = pd.DataFrame([
    {
        "Modelo": "LGBM",
        "Parâmetro": "n_estimators",
        "Valores testados": "{400, 600, 800}",
        "Escolhido": 800,
        "Observação": "trade-off tempo vs erro"
    },
    {
        "Modelo": "LGBM",
        "Parâmetro": "learning_rate",
        "Valores testados": "{0.05, 0.03}",
        "Escolhido": 0.03,
        "Observação": "mais estável"
    },
    {
        "Modelo": "LGBM",
        "Parâmetro": "num_leaves / max_depth",
        "Valores testados": "num_leaves {31, 63}; max_depth {-1, 6}",
        "Escolhido": "63 / 6",
        "Observação": "complexidade moderada"
    },
    {
        "Modelo": "LGBM",
        "Parâmetro": "min_child_samples",
        "Valores testados": "{5, 10}",
        "Escolhido": 5,
        "Observação": "evita overfitting mantendo generalização"
    },
    {
        "Modelo": "LGBM",
        "Parâmetro": "subsample / colsample_bytree",
        "Valores testados": "{0.9}",
        "Escolhido": "0.9 / 0.9",
        "Observação": "regularização leve"
    },
    {
        "Modelo": "SARIMAX",
        "Parâmetro": "order (p,d,q)",
        "Valores testados": "{(1,1,1), (1,1,2), (2,1,2)}",
        "Escolhido": "(1,1,2)",
        "Observação": "menor erro no holdout"
    },
    {
        "Modelo": "SARIMAX",
        "Parâmetro": "seasonal_order (P,D,Q,s)",
        "Valores testados": "{(1,0,1,4), (1,0,2,4)}",
        "Escolhido": "(1,0,1,4)",
        "Observação": "sazonalidade trimestral (s=4)"
    },
    {
        "Modelo": "SARIMAX",
        "Parâmetro": "Exógenas",
        "Valores testados": "{caged_saldo_tri, caged_roll3_asinh, selic_tri(opc.)}",
        "Escolhido": "{caged_saldo_tri, caged_roll3_asinh}",
        "Observação": "CAGED captura variações do emprego formal"
    }
])
hp



---
### Apêndice (opcional)

Blocos auxiliares (ex.: sliders de avaliação) devem ser inseridos aqui para não poluir a execução principal.
